In [1]:
# Before collaborative filtering, do train_test split 

In [2]:
!pip install turicreate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.0 MB 11 kB/s 
     |████████████████████████████████| 322 kB 15.4 MB/s 
     |████████████████████████████████| 3.6 MB 33.5 MB/s 
     |████████████████████████████████| 86.4 MB 1.7 MB/s 
     |████████████████████████████████| 3.5 MB 51.1 MB/s 
     |████████████████████████████████| 18.3 MB 275 kB/s 
     |████████████████████████████████| 2.9 MB 45.6 MB/s 
     |████████████████████████████████| 20.1 MB 79.4 MB/s 
     |████████████████████████████████| 3.8 MB 52.2 MB/s 
     |████████████████████████████████| 449 kB 49.2 MB/s 
     |████████████████████████████████| 50 kB 5.3 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=2ebb9d5411a692f6789fb681bc8f1f0ec6680ccbf5275bf2ee7ad940bda712ae
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c

In [3]:
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd 
import turicreate as tc 
from turicreate import SFrame
from turicreate.toolkits.recommender.util import precision_recall_by_user
import numpy as np
import pickle
import statistics
from sklearn.metrics import accuracy_score

In [4]:
import os
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/MyDrive/Web Mining"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive',force_remount=True)  
os.chdir("/content/drive/MyDrive/Web Mining")

Mounted at /content/drive


In [8]:

allmovies = pd.read_csv('/content/drive/MyDrive/Web Mining/Unused/ml-1m/movies.dat', sep="::",names=['MovieID', 'Title', 'Genres'],encoding='latin-1')
ratings = pd.read_csv('/content/drive/MyDrive/Web Mining/Unused/ml-1m/ratings.dat', sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], skipinitialspace=True)
ratings = ratings.drop('Timestamp', axis=1) 
# users = pd.read_csv('data/users.dat', sep="::", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"],skipinitialspace=True) 
movie_final_full_dataset = pd.read_csv('Final Data/movie_final_full_dataset.csv')
groups = pd.read_csv("./Pipeline/users_grouped_by3.csv", skipinitialspace=True)
print(groups["Group"].nunique())

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


102


In [9]:
movies=pd.merge(movie_final_full_dataset,allmovies[['MovieID']],on=["MovieID"],how='left')
ratings=pd.merge(ratings,movie_final_full_dataset[['MovieID']],on=["MovieID"],how='right')

In [10]:
allusers= ratings['UserID']
uniqueUsers = list(dict.fromkeys(allusers))
user2movie={}
for user in uniqueUsers:
    user2movie[user]=set(ratings.loc[(ratings['UserID'] == user)]['MovieID'])

### Split the test set to be the common movies for each group and the train set is the rest of the movies

In [11]:
groupsID=groups['Group'].unique()
group2test={}
for g in groupsID:
  users=list(groups.loc[groups['Group'] == g]['UserID'])
  commonIds=user2movie[users[0]]
  for u in users:
    commonIds=commonIds.intersection(user2movie[u])
  group2test[g]=commonIds


In [12]:
# filter on the users on the groups only
ratings=ratings.loc[ratings['UserID'].isin(groups['UserID'])]
movies=movies.loc[movies['MovieID'].isin(ratings['MovieID'])] # filter to get only the movies related to our users
# filter on the train records only
ratings_train=ratings.copy()
for g in groupsID:
  users=list(groups.loc[groups['Group'] == g]['UserID'])
  id2drop=group2test[g]
  for u in users:
    i = ratings_train[((ratings_train.UserID == u) & ratings_train['MovieID'].isin(id2drop))].index
    ratings_train.drop(i,inplace=True)
ratings_train.reset_index(drop=True,inplace=True)
ratings_train

,UserID,MovieID,Rating
0,123,1,3
1,148,1,5
2,169,1,5
3,195,1,5
4,302,1,4
...,...,...,...
168018,5333,3952,4
168019,5359,3952,5
168020,5682,3952,3
168021,5812,3952,4


# prepare the model

In [13]:
# actions = ratings 
ratings_tc = SFrame(data = ratings_train)
ratings_tc.head() 

UserID,MovieID,Rating
123,1,3
148,1,5
169,1,5
195,1,5
302,1,4
308,1,4
329,1,3
411,1,5
424,1,4
438,1,5


In [14]:
ratings_tc.show()

Materializing SFrame

In [15]:

train_tc, valid_tc = tc.recommender.util.random_split_by_user(ratings_tc, user_id='UserID', item_id='MovieID', item_test_proportion=0.65)
model= tc.ranking_factorization_recommender.create(train_tc, 'UserID', 'MovieID', target='Rating', ranking_regularization=0.1, unobserved_rating_value = 1)
# Final objective value: 0.746178
#        Final training RMSE: 0.520819

Preparing data set.

Data has 58795 observations with 306 users and 3202 items.

Data prepared in: 0.054713s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.587558

Final training RMSE: 0.344515

In [16]:
# evaluate - model without user data, chosen by RC
model.evaluate(valid_tc)
# 'rmse_overall': 1.239300118353719


Precision and recall summary statistics by cutoff
+--------+---------------------+-----------------------+
| cutoff |    mean_precision   |      mean_recall      |
+--------+---------------------+-----------------------+
|   1    | 0.42810457516339867 |  0.001230670401079516 |
|   2    |  0.3937908496732026 |  0.002263305648364392 |
|   3    | 0.38453159041394336 | 0.0033379045701655164 |
|   4    |  0.3831699346405229 |  0.004410145773035721 |
|   5    |  0.3673202614379086 |  0.005275168107419106 |
|   6    | 0.35620915032679734 |  0.006132054109659677 |
|   7    |  0.3566760037348272 |  0.007164334122779382 |
|   8    |  0.3574346405228757 |  0.008192902385674434 |
|   9    |  0.3493100944081335 |  0.00901851468775091  |
|   10   | 0.34705882352941186 |  0.009947369992069726 |
+--------+---------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.3101105400974933

Per User RMSE (best)
+--------+--------------------+-------+
| UserID |        rmse        | 

{'precision_recall_by_user': Columns:
 	UserID	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 5508
 
 Data:
 +--------+--------+---------------------+-----------------------+-------+
 | UserID | cutoff |      precision      |         recall        | count |
 +--------+--------+---------------------+-----------------------+-------+
 |  123   |   1    |         0.0         |          0.0          |  334  |
 |  123   |   2    |         0.0         |          0.0          |  334  |
 |  123   |   3    |  0.3333333333333333 | 0.0029940119760479044 |  334  |
 |  123   |   4    |         0.25        | 0.0029940119760479044 |  334  |
 |  123   |   5    |         0.2         | 0.0029940119760479044 |  334  |
 |  123   |   6    | 0.16666666666666666 | 0.0029940119760479044 |  334  |
 |  123   |   7    | 0.14285714285714285 | 0.0029940119760479044 |  334  |
 |  123   |   8    |         0.25        |  0.005988023952095809 |  334  |
 |  123   |   9    |  0.3333333333333333 | 

# predict for groups and evaluate

In [20]:
def evaluate(true_values,predictions,t):
# do aggregation 
# Average 
  if t=="average":
    yp=np.sum(predictions, axis=0)
    yp=yp/len(predictions)
    yt=np.sum(true_values, axis=0)
    yt=yt/len(true_values)
  elif t=="least misery":
    yp=np.amin(predictions, axis=0)
    yt=np.amin(true_values, axis=0)
  elif t=="most pleasure":
    yp=np.amax(predictions, axis=0)
    yt=np.amax(true_values, axis=0)
  elif t=="average without misery":
    yp=np.sum(predictions, axis=0)
    min=np.amin(predictions, axis=0)
    yp=yp-min
    yp=yp/(len(predictions)-1)
    yt=np.sum(true_values, axis=0)
    min=np.amin(true_values, axis=0)
    yt=yt-min
    yt=yt/(len(true_values)-1)
  else: 
    print("only these options available: [average,least misery,most pleasure,average without misery]")
  # if the score is more than 3 then he likes the movie
  yt=np.where(yt < 4, 0, 1)
  yp=np.where(yp < 4, 0, 1)
  return accuracy_score(np.round(yt),np.round(yp))

In [23]:
def make_prediction(userID,groupID):
  sf = tc.SFrame({'UserID': [userID]*len(group2test[groupID]),
             'MovieID': group2test[groupID]})
  return model.predict(sf)
def group_predict(t):
  results_acc=[]
  results_ndcg=[]
  group2predictions={}
  group2gs={}
  for g in groupsID:
    users=list(groups.loc[groups['Group'] == g]['UserID'])
    true_values=[]
    predictions=[]
    for u in users:
      true_values.append(list(ratings[((ratings.UserID == u) & ratings['MovieID'].isin( group2test[g]))]['Rating']))
      predictions.append(list(make_prediction(u,g)))
    group2predictions[g]=predictions
    group2gs[g]=true_values
    results=evaluate(true_values,predictions,t)
    results_acc.append(results)
    # results_ndcg.append(results[1])
  print(t,statistics.mean(results_acc))

In [24]:
group_predict('average')

average 0.5794834986343905


In [ ]:
group_predict('average')
group_predict('least misery')
group_predict('most pleasure')
group_predict('average without misery')

0.5797557695888073

In [ ]:

# path="/content/drive/MyDrive/Web Mining/Pipline/"
# pickle.dump( group2predictions, open( path+'cf_predictions.p', "wb" ) )
# pickle.dump( group2gs, open( path+'true_value.p', "wb" ) )